# 토큰화 (Tokenization)

- 문장이나 단어를 더 작은 단위로 나누어 분석 가능한 단위(토큰, Token)으로 변환하는 과정
- 토큰의 단위가 상황에 따라 다르지만, 보통 의미있는 혹은 처리하는 단위로써 토큰 정의
- 자연어 처리에서 크롤링, 데이터 수집 등으로 얻은 코퍼스 데이터는 정제되지 않은 경우가 많은데 이를 사용 용도에 맞게 토큰화, 정제, 정규화하는 과정이 필요

**토큰화 목적**

- 문법적 구조 이해
- 유연한 데이터 활용

In [1]:
!pip install transformers spacy "kss==5.0.0" tensorflow 

In [2]:
# 3.12 버전의 python 가상환경
# python3.12 -m venv (가상환경명)
# python3.12 -m venv nlp_venv

In [3]:
import nltk

nltk.download('punkt')          # 문장/단어 토큰화용 리소스
nltk.download('punkt_tab')      # punkt 관련 추가 테이블 리소스


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
text = "NLP is fascinating. It has many applications in real-world scenarios."

In [5]:
# NLTK 토큰화
import nltk

nltk.word_tokenize(text)    # text를 단어(토큰) 리스트로 분리

['NLP',
 'is',
 'fascinating',
 '.',
 'It',
 'has',
 'many',
 'applications',
 'in',
 'real-world',
 'scenarios',
 '.']

In [6]:
print(nltk.sent_tokenize(text)) # text를 문장 리스트로 분리

['NLP is fascinating.', 'It has many applications in real-world scenarios.']


In [7]:
for sent in nltk.sent_tokenize(text):   # 문장 단위로 순회
    print(nltk.word_tokenize(sent))     # 각 문장을 단어 토큰으로 분리해 출력

['NLP', 'is', 'fascinating', '.']
['It', 'has', 'many', 'applications', 'in', 'real-world', 'scenarios', '.']


### Subword Tokenization

- BertTokenizer
    - 단어를 부분 단위로 쪼개어 희귀하거나, 새로운 단어도 부분적으로 표현 할 수 있도록 한다.  
    - 어휘 크기를 줄이고 다양한 언어 패턴 학습 가능  

In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 소문자 기준

#word = 'happy'
word = 'unhappiness'
subwords = tokenizer.tokenize(word)
subwords

c:\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['un', '##ha', '##pp', '##iness']

### - BERT의 subword 토큰화  
    - 단어를 사전에 있는 조각(Subword) 단위로 쪼개어 표현  
    - 처음 조각 그대로, 단어 중간/뒤의 단어는 ##를 붙여서 표현한다.  
    - 통째로 완전한 단어가 없으면 가장 긴 조각을 먼저 찾아 쪼갠다.
    - 끝까지 조각이 안맞으면 [UNK] (unknown) 토큰으로 처리한다.  
    - 처음 보는 단어를 줄이고, 희귀 단어를 조각 조합으로 찾을 수 있도록 한다. 
    

### - 문자 단위 토큰화

In [9]:
list(word)

['u', 'n', 'h', 'a', 'p', 'p', 'i', 'n', 'e', 's', 's']

In [ ]:
# 정규표현식으로 단어 추출
import re

text = 'Time files like an arrow; fruit files like a banana'
re.findall(r"\b\w+\b", text)    # 단어 경계 기준으로 단어(문자/숫자/_)를 전부 추출

['Time',
 'files',
 'like',
 'an',
 'arrow',
 'fruit',
 'files',
 'like',
 'a',
 'banana']

\b : 단어 경계. \b ... \b 로 감싸면 안에 있는 패턴 형태만 추출  
\w : 문자/숫자_ 1개 (\w+ : 문자 덩어리)  

In [ ]:
# wordPuncTokenizer(단어/구두점으로 토큰을 구분. ',- 포함 단어도 분리) 와 word_tokenize
from nltk.tokenize import WordPunctTokenizer, word_tokenize

text = "Don't hesitate to use well-being practices for self-care."
word_punct_tokenizer = WordPunctTokenizer()
print(word_punct_tokenizer.tokenize(text))
print(word_tokenize(text))

['Don', "'", 't', 'hesitate', 'to', 'use', 'well', '-', 'being', 'practices', 'for', 'self', '-', 'care', '.']
['Do', "n't", 'hesitate', 'to', 'use', 'well-being', 'practices', 'for', 'self-care', '.']


WordPunctTokenizer 는 ',- 같은 구두점 기준으로 더 잘개 쪼갠다.  
word_tokenize 는 상대적으로 자연스러운 단어 단위로 토큰화

In [12]:
# TreebankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer, word_tokenize

text = '''
COVID-19(전염병), Dr.Smith(의사), NASA(우주항공국) 등 특정 기관이나 명칭이 있다.
특수 문자 또한 태그 <br>, 가격 $100.50, 2025/08/18 날짜 표현에 사용될 수 있다.
이러한 경우, $100.50을 하나의 토큰으로 유지할 필요가 있다.
'''

treebank_word_tokenizer = TreebankWordTokenizer()
print(treebank_word_tokenizer.tokenize(text))                   # Treebank 규칙 기반 토크나이저
print(word_tokenize(text))                                      # NLTK 기본 토큰화

['COVID-19', '(', '전염병', ')', ',', 'Dr.Smith', '(', '의사', ')', ',', 'NASA', '(', '우주항공국', ')', '등', '특정', '기관이나', '명칭이', '있다.', '특수', '문자', '또한', '태그', '<', 'br', '>', ',', '가격', '$', '100.50', ',', '2025/08/18', '날짜', '표현에', '사용될', '수', '있다.', '이러한', '경우', ',', '$', '100.50을', '하나의', '토큰으로', '유지할', '필요가', '있다', '.']
['COVID-19', '(', '전염병', ')', ',', 'Dr.Smith', '(', '의사', ')', ',', 'NASA', '(', '우주항공국', ')', '등', '특정', '기관이나', '명칭이', '있다', '.', '특수', '문자', '또한', '태그', '<', 'br', '>', ',', '가격', '$', '100.50', ',', '2025/08/18', '날짜', '표현에', '사용될', '수', '있다', '.', '이러한', '경우', ',', '$', '100.50을', '하나의', '토큰으로', '유지할', '필요가', '있다', '.']


하이픈, 괄호, <br>, $100.50, 날짜 등이 있을때 추가적으로 차이가 발생할 수 있다.  
현재 예문에서는 있다. 정도 차이  

### 한국어 토큰화

In [13]:
import kss  #한국어 문장 분리 라이브러리(Korean Sentence Splitter)
import warnings
warnings.filterwarnings("ignore")


text = "시간적 배경은 1920년대의 겨울로, 공간적 배경은 경성부. 주인공이자 인력거꾼 김 첨지의 아내는 병에 걸린 지 1달 가량이 지나 있었다. 아내는 단 한 번도 약을 먹어본 적이 없는데, 그 이유는 '병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다'는 김 첨지의 신조 때문으로 나오지만 사실 이건 핑계고, 약을 살 돈도 벌지 못하고 있었다는 이유가 더 크다."

kss.split_sentences(text)   # 문장 단위로 분리

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363



['시간적 배경은 1920년대의 겨울로, 공간적 배경은 경성부.',
 '주인공이자 인력거꾼 김 첨지의 아내는 병에 걸린 지 1달 가량이 지나 있었다.',
 "아내는 단 한 번도 약을 먹어본 적이 없는데, 그 이유는 '병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다'는 김 첨지의 신조 때문으로 나오지만 사실 이건 핑계고, 약을 살 돈도 벌지 못하고 있었다",
 '는 이유가 더 크다.']

### 품사 태깅

**pos_tag**

pos_tag는 자연어 처리(NLP)에서 단어에 품사를 태깅하는 함수로, 주로 NLTK와 같은 라이브러리에서 사용된다.

- nltk pos_tag() 주요 품사 태깅<br>

1. **NN (Noun, Singular)**  
   단수 명사를 나타낸다. 하나의 사물이나 개념을 지칭한다.  
   예시: "cat", "book", "apple"

2. **NNS (Noun, Plural)**  
   복수 명사를 나타낸다. 두 개 이상의 사물이나 개념을 지칭한다.  
   예시: "cats", "books", "apples"

3. **NNP (Proper Noun, Singular)**  
   단수 고유 명사를 나타낸다. 특정한 사람, 장소 또는 조직의 이름을 지칭한다.  
   예시: "Alice", "London", "NASA"

4. **NNPS (Proper Noun, Plural)**  
   복수 고유 명사를 나타낸다. 두 개 이상의 특정한 사람, 장소 또는 조직의 이름을 지칭한다.  
   예시: "Smiths", "United Nations"

5. **VB (Verb, Base Form)**  
   동사의 원형을 나타낸다. 일반적으로 현재 시제와 함께 사용된다.  
   예시: "run", "eat", "play"

6. **VBD (Verb, Past Tense)**  
   동사의 과거형을 나타낸다.  
   예시: "ran", "ate", "played"

7. **VBG (Verb, Gerund or Present Participle)**  
   동명사 또는 현재 분사를 나타낸다. 일반적으로 "-ing" 형태이다.  
   예시: "running", "eating", "playing"

8. **VBN (Verb, Past Participle)**  
   동사의 과거 분사형을 나타낸다. 주로 완료 시제와 함께 사용된다.  
   예시: "run" (as in "has run"), "eaten", "played"

9. **VBZ (Verb, 3rd Person Singular Present)**  
   3인칭 단수 현재형 동사를 나타낸다. 주어가 3인칭 단수일 때 사용된다.  
   예시: "runs", "eats", "plays"

10. **JJ (Adjective)**  
    형용사를 나타낸다. 명사를 수식하여 그 특성을 설명한다.  
    예시: "big", "blue", "happy"

11. **JJR (Adjective, Comparative)**  
    비교급 형용사를 나타낸다. 두 개의 대상을 비교할 때 사용된다.  
    예시: "bigger", "bluer", "happier"

12. **JJS (Adjective, Superlative)**  
    최상급 형용사를 나타낸다. 세 개 이상의 대상을 비교할 때 사용된다.  
    예시: "biggest", "bluest", "happiest"

13. **RB (Adverb)**  
    부사를 나타낸다. 동사, 형용사 또는 다른 부사를 수식한다.  
    예시: "quickly", "very", "well"

14. **RBR (Adverb, Comparative)**  
    비교급 부사를 나타낸다. 두 개의 대상을 비교할 때 사용된다.  
    예시: "more quickly", "better"

15. **RBS (Adverb, Superlative)**  
    최상급 부사를 나타낸다. 세 개 이상의 대상을 비교할 때 사용된다.  
    예시: "most quickly", "best"

16. **IN (Preposition or Subordinating Conjunction)**  
    전치사 또는 종속 접속사를 나타낸다. 명사와의 관계를 나타내거나 종속절을 시작한다.  
    예시: "in", "on", "because"

17. **DT (Determiner)**  
    한정사를 나타낸다. 명사의 수와 상태를 정의한다.  
    예시: "the", "a", "some"

18. **PRP (Personal Pronoun)**  
    인칭 대명사를 나타낸다. 사람, 사물 등을 대체할 때 사용된다.  
    예시: "I", "you", "he", "they"

19. **PRP$ (Possessive Pronoun)**  
    소유 대명사를 나타낸다. 소유 관계를 나타낸다.  
    예시: "my", "your", "his", "their"

In [14]:
# NLTK 품사 태그 리소스 다운로드
nltk.download('averaged_perceptron_tagger')     # NLTK 기본 POS TAGGER 모델
nltk.download('averaged_perceptron_tagger_eng') # 영어 버전

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [15]:
from nltk.tag import pos_tag

text = 'Time flies like an arrow'   
tokens = word_tokenize(text)        # 단어 토큰화
pos_tags = pos_tag(tokens)          # 각 토큰에 품사 태그 부여 : (토큰,품사태그) 형태의 리스트
pos_tags

[('Time', 'NNP'),
 ('flies', 'NNS'),
 ('like', 'IN'),
 ('an', 'DT'),
 ('arrow', 'NN')]

**spacy 주요 품사 태깅**

| 태그 | 설명                 | 예시                 |
|------|----------------------|----------------------|
| ADJ  | 형용사               | big, nice           |
| ADP  | 전치사               | in, to, on          |
| ADV  | 부사                 | very, well          |
| AUX  | 조동사               | is, have (조동사로 사용될 때) |
| CONJ | 접속사               | and, or             |
| DET  | 한정사/관사          | the, a              |
| INTJ | 감탄사               | oh, wow             |
| NOUN | 명사                 | dog, table          |
| NUM  | 숫자                 | one, two, 3         |
| PART | 소사                 | 'to' (to fly에서), not |
| PRON | 대명사               | he, she, it         |
| PROPN| 고유명사             | John, France        |
| PUNCT| 구두점               | ., !, ?             |
| SCONJ| 종속 접속사          | because, if         |
| SYM  | 기호                 | $, %, @             |
| VERB | 동사                 | run, eat            |
| X    | 알 수 없는 품사       | 외국어 단어, 잘못된 형식 |

In [16]:
import spacy 

spacy.cli.download('en_core_web_sm')        # spacy 영어 모델 다운로드
spacy.nlp = spacy.load('en_core_web_sm')    # 모델 로드

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import spacy

# 영어 모델 로드
spacy_nlp = spacy.load("en_core_web_sm")

text = 'Time flies like an arrow'
tokens = spacy_nlp(text)

tokens


Time flies like an arrow

In [19]:
for token in tokens:
    print(token.text, ':', token.pos_)

Time : NOUN
flies : VERB
like : ADP
an : DET
arrow : NOUN


### KoNLPy

- 한국어 자연어 처리를 위한 라이브러리
- 형태소 분석, 품사 태깅, 텍스트 전처리 등 기능 지원
- 여러 형태소 분석기 중 적합한 분석기 선택 가능

In [25]:
# Okt 형태소 분석
from konlpy.tag import Okt

text = '오늘 점심은 뭐 먹을까? 맛있는 게 뭐지?'
okt = Okt(jvmpath=r"C:\Program Files\Java\jdk-21\bin\server\jvm.dll")


morphs = okt.morphs(text)   # 형태소 단위로 분리
morphs

['오늘', '점심', '은', '뭐', '먹을까', '?', '맛있는', '게', '뭐', '지', '?']

In [26]:
pos_tags = okt.pos(text)
pos_tags

[('오늘', 'Noun'),
 ('점심', 'Noun'),
 ('은', 'Josa'),
 ('뭐', 'Noun'),
 ('먹을까', 'Verb'),
 ('?', 'Punctuation'),
 ('맛있는', 'Adjective'),
 ('게', 'Noun'),
 ('뭐', 'Noun'),
 ('지', 'Josa'),
 ('?', 'Punctuation')]

In [ ]:
# 명사 추출
nouns = okt.nouns(text) # 문장에서 명사만 추출
nouns

['오늘', '점심', '뭐', '게', '뭐']